# Assistant-03<br/>Knowledge Retrieval


### Get an OpenAI client

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [2]:
import common
import shelve
import time

## Get an OpenAI client
client = common.get_openai_client()

In [14]:
def upload_file(path):
    # Upload a file with an "assistants" purpose
    file = client.files.create(file=open(path, "rb"), purpose="assistants")
    return file

file = upload_file("../data/faq.txt")
print(file)


FileObject(id='assistant-r6QKRp4qLpz4q4LyrU6mejrh', bytes=2606, created_at=1705076540, filename='faq.txt', object='file', purpose='assistants', status='processed', status_details=None)


In [15]:
def create_assistant(file):
    """
    You currently cannot set the temperature for Assistant via the API.
    """
    print(common.gpt_deployment_name)    
    assistant = client.beta.assistants.create(
        name="Contoso Store Assistant",
        instructions="""You are a customer support chatbot. 
Use your knowledge base to best respond to customer queries.""",
        tools=[{"type": "retrieval"}],
        model=common.gpt_deployment_name,
        file_ids=[file.id]
    )    
    return assistant


assistant = create_assistant(file)

gpt-4-turbo-test


BadRequestError: Error code: 400 - {'error': {'message': 'Failed to index file: File assistant-r6QKRp4qLpz4q4LyrU6mejrh not found.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
# --------------------------------------------------------------
# Thread management
# --------------------------------------------------------------
def check_if_thread_exists(wa_id):
    with shelve.open("threads_db") as threads_shelf:
        return threads_shelf.get(wa_id, None)


def store_thread(wa_id, thread_id):
    with shelve.open("threads_db", writeback=True) as threads_shelf:
        threads_shelf[wa_id] = thread_id


# --------------------------------------------------------------
# Generate response
# --------------------------------------------------------------
def generate_response(message_body, wa_id, name):
    # Check if there is already a thread_id for the wa_id
    thread_id = check_if_thread_exists(wa_id)

    # If a thread doesn't exist, create one and store it
    if thread_id is None:
        print(f"Creating new thread for {name} with wa_id {wa_id}")
        thread = client.beta.threads.create()
        store_thread(wa_id, thread.id)
        thread_id = thread.id

    # Otherwise, retrieve the existing thread
    else:
        print(f"Retrieving existing thread for {name} with wa_id {wa_id}")
        thread = client.beta.threads.retrieve(thread_id)

    # Add message to thread
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message_body,
    )

    # Run the assistant and get the new message
    new_message = run_assistant(thread)
    print(f"To {name}:", new_message)
    return new_message

In [ ]:
# --------------------------------------------------------------
# Run assistant
# --------------------------------------------------------------
def run_assistant(thread):
    # Retrieve the Assistant
    assistant = client.beta.assistants.retrieve("asst_7Wx2nQwoPWSf710jrdWTDlfE")

    # Run the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    # Wait for completion
    while run.status != "completed":
        # Be nice to the API
        time.sleep(0.5)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Retrieve the Messages
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    new_message = messages.data[0].content[0].text.value
    print(f"Generated message: {new_message}")
    return new_message

In [ ]:
# --------------------------------------------------------------
# Test assistant
# --------------------------------------------------------------
new_message = generate_response("What's the check in time?", "123", "John")

new_message = generate_response("What's the pin for the lockbox?", "456", "Sarah")

new_message = generate_response("What was my previous question?", "123", "John")

new_message = generate_response("What was my previous question?", "456", "Sarah")